# Test Tabular Regression Model 

## Setup

In [9]:
# Fastai is required by eb_ml_battery_lib
!pip install fastai==2.5.3 -q
# import
import pandas as pd
import numpy as np
import sys
from fastai.tabular import *
from fastai.tabular.all import *
# Load EB_ML python libraries
# The following libraries are used in this notebook and should be installed in your local machine before running this notebook.
# eb_colab_utils.py
# eb_ml_battery_lib.py
# eb_ml_utils.py

# path to load external *.py files used in this notebook
# Note: in Google Colab virtual machine you shoud copy the files in "/content" folder after BEFORE running this notebook's cell
external_python_file_path='../src'
sys.path.append(external_python_file_path)


from eb_ml_colab_utils import get_root_path
from eb_ml_battery_lib import build_EIS_tabular_learner_rectangular,load_soc_dataset,get_EIS_tabular_dataset_rectangular
from eb_ml_utils import build_tabular_learner

You should consider upgrading via the '/Users/emanuelebuchicchio/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


## Load EIS tabular regression model trained with measure on  battery 2,3,5

In [10]:
#configuration dictionary
config ={}

# Root working folder (local or Google Drive)
# config['ROOT_DIR'] = get_root_path("batterie")
config['ROOT_DIR'] = get_root_path("..")  

# Folder with dataset in CSV format
#config['DATASETS_DIR'] = config['ROOT_DIR']+"/datasets"
config['DATASETS_DIR'] = config['ROOT_DIR']+"/datasets/EIS-vs-SOC-May2022"

# List of SoC level into dataset
#config['soc_list']=['100','090','080','070','060','050','040','030','020','010']
config['soc_list']=['100','090','080','070','060','050','040','030','020','010']

# Data acquisition files to load from dateset folder
measure_list=["02_4","02_5","02_6","02_7","02_8","02_9","03_4","03_5","03_6","03_7","03_8","03_9","05_3","05_4","05_5","05_6","05_7","05_8"]


# Build FastAI's DataLoader and Learner objects
learn= build_EIS_tabular_learner_rectangular(config,measure_list)

#Load Model weights from file saved after training with battery 02 dataset
learn.load('EIS_tabular_regression_2-3-5_polar')

NOT running on COLAB


## Tabular Learner NN Architecture

In [11]:
learn.model

TabularModel(
  (embeds): ModuleList()
  (emb_drop): Dropout(p=0.0, inplace=False)
  (bn_cont): BatchNorm1d(28, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layers): Sequential(
    (0): LinBnDrop(
      (0): Linear(in_features=28, out_features=300, bias=False)
      (1): ReLU(inplace=True)
      (2): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): LinBnDrop(
      (0): Linear(in_features=300, out_features=200, bias=False)
      (1): ReLU(inplace=True)
      (2): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (2): LinBnDrop(
      (0): Linear(in_features=200, out_features=100, bias=False)
      (1): ReLU(inplace=True)
      (2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (3): LinBnDrop(
      (0): Linear(in_features=100, out_features=50, bias=False)
      (1): ReLU(inplace=True)
      (2): BatchNorm1d(50, eps=1e-05, momentum=0

## Get Prediction from validation dataset

In [12]:
from sklearn.metrics import mean_squared_error
[y_pred,y_true] =learn.get_preds()
mean_squared_error(y_true,y_pred,squared=False)

2.5942085

## Load measure on battery 06 as test dataset

In [13]:
test_measure_list=["06_3","06_4","06_5","06_6","06_7","06_8"]
test_dataset,feature_col_names=load_soc_dataset(test_measure_list,config["soc_list"],config['DATASETS_DIR'])

# Put complex number in rectangular format
test_dataset_rect,feature_col_names_rect=get_EIS_tabular_dataset_rectangular(test_dataset,feature_col_names)

# Create a new Data Loader object to load test dataset
test_dl = learn.dls.test_dl(test_dataset_rect)


/Users/emanuelebuchicchio/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:7134: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return concat(


# Get Prediction on battery 06 test dataset

In [14]:
from sklearn.metrics import mean_squared_error
[y_pred,y_true] =learn.get_preds(dl=test_dl)

#RMS Error predizione modello adestrato du dati batteria due eseguito su dati batteria 
mean_squared_error(y_true,y_pred,squared=False)

3.5789697